In [1]:
import time
import math
import builtins  # para garantir o uso do max() nativo do Python
from pyspark.sql.functions import col
from pyspark import StorageLevel
from delta.tables import DeltaTable

# ============================================================
# 0. Configurações de Segurança e Performance
# ============================================================

# Desliga o optimizeWrite para termos controle total sobre a quantidade de arquivos.
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "false")

# Mantém o AQE ligado (sempre bom).
spark.conf.set("spark.sql.adaptive.enabled", "true")

# Caminho da tabela de destino
path_destino = (
    "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/"
    "lk_departamento_pessoal.Lakehouse/Tables/tab_gold_dim_funcionario_ultima_posicao_mes"
)

# Aumentamos o alvo. Se vamos salvar "milhares", geralmente queremos 
# agrupar tudo em 1 ou poucos arquivos. 500k é um bom número para Delta.
LINHAS_POR_ARQUIVO = 500_000 

RETENCAO_VACUUM_HORAS = 1

# ============================================================
# 1. Leitura e Materialização
# ============================================================

print("1. Iniciando leitura e filtro...")
start_read = time.time()

# Leitura da Tabela Gigante (Milhões de linhas)
df_historico = spark.table("tab_gold_dim_funcionario_historico")

# O Filtro que reduz drasticamente o volume (O "Funil")
df_filtrado = df_historico.filter(col("RK_MES") == 1)

# Persistimos para garantir que a contagem e a escrita usem os mesmos dados
# sem reler a tabela gigante duas vezes.
df_cached = df_filtrado.persist(StorageLevel.MEMORY_AND_DISK)

# Força o processamento para sabermos o tamanho do resultado
total_linhas = df_cached.count()

end_read = time.time()
print(f"   Leitura concluída em {end_read - start_read:.2f} segundos.")
print(f"   Total de linhas resultantes (saída): {total_linhas}")

# ============================================================
# 2. Escrita com COALESCE (A Solução do Efeito Funil)
# ============================================================

if total_linhas > 0:
    # Calcula quantos arquivos precisamos (provavelmente será 1)
    num_arquivos_finais = builtins.max(1, math.ceil(total_linhas / LINHAS_POR_ARQUIVO))

    print(
        f"2. Iniciando gravação Otimizada (Coalesce) em {num_arquivos_finais} arquivo(s)..."
    )

    (
        df_cached
            # ============================================================
            # A MUDANÇA CRÍTICA: COALESCE
            # ============================================================
            # Diferente do repartition(), o coalesce NÃO embaralha os dados
            # pela rede. Ele apenas diz às partições: "Juntem-se!".
            # É perfeito para quando você tem muitas partições de entrada
            # e quer poucas de saída.
            .coalesce(num_arquivos_finais)
            .write
            .format("delta")
            .mode("overwrite")
            .option("overwriteSchema", "true")
            .save(path_destino)
    )

    print("   Gravação concluída com sucesso.")
else:
    print("   Nenhum dado encontrado para gravar.")

# Libera recursos
df_cached.unpersist()

# ============================================================
# 3. Limpeza (VACUUM)
# ============================================================

print("3. Executando limpeza física (VACUUM)...")

spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")
delta_table = DeltaTable.forPath(spark, path_destino)
delta_table.vacuum(RETENCAO_VACUUM_HORAS)
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "true")

print("Processo finalizado.")

StatementMeta(, 90a291d7-6f36-41d4-ad68-272fe51ac41c, 3, Finished, Available, Finished)

1. Iniciando leitura e filtro...
   Leitura concluída em 34.41 segundos.
   Total de linhas resultantes (saída): 827277
2. Iniciando gravação Otimizada (Coalesce) em 2 arquivo(s)...
   Gravação concluída com sucesso.
3. Executando limpeza física (VACUUM)...
Processo finalizado.
